In [ ]:
%matplotlib inline

Adapted from:
* https://github.com/miykael/workshop_pybrain/ 
* https://nilearn.github.io/auto_examples/07_advanced/plot_bids_analysis.html
* https://nilearn.github.io/auto_examples/07_advanced/plot_bids_analysis.html

# Nilearn GLM: statistical analyses of MRI in Python


`Nilearn`'s `GLM/stats` module allows fast and easy MRI statistical analysis.

It leverages `Nibabel` and other Python libraries from the Python scientific stack like `Scipy`, `Numpy` and `Pandas`.

In this tutorial, we're going to explore `nilearn's GLM` functionality by analysing a sample dataset of language localiser task. We will analyse 1) a single subject data and 2) ten subject group level example using a General Linear Model (GLM).  

## Fetch example BIDS dataset
We download a simplified `BIDS` dataset made available for illustrative
purposes. It contains only the necessary
information to run a statistical analysis using `Nilearn`. The raw data
subject folders only contain `bold.json` and `events.tsv` files, while the
`derivatives` folder includes the preprocessed files `preproc.nii` and the
`confounds.tsv files.`

In [ ]:
from nilearn.datasets import fetch_language_localizer_demo_dataset
data_dir, _ = fetch_language_localizer_demo_dataset()

Here is the location of the dataset on disk.



In [ ]:
print(data_dir)

We can also use, e.g., `seedir` to explore the contents of this example dataset. 

In [ ]:
import seedir as sd
sd.seedir(data_dir, style='emoji')

## GLM on a single subject

### Specifying the experimental paradigm
We must now provide a description of the experiment, that is, define the
timing of the task and rest periods. This is typically
provided in an `events.tsv file`.

In [ ]:
from os.path import join
import pandas as pd
events = pd.read_table(join(data_dir, 'sub-01/func/sub-01_task-languagelocalizer_events.tsv'))
print(events)

### Performing the GLM analysis
It is now time to create and estimate a `FirstLevelModel` object, that will generate the *design matrix* using the  information provided by the ``events`` object.

In [ ]:
from nilearn.glm.first_level import FirstLevelModel

There are a lot of important parameters one needs to define within a `FirstLevelModel` and the majority of them will have a prominent influence on your results. Thus, make sure to check them before running your model:

In [ ]:
FirstLevelModel?

In [ ]:
fmri_glm = FirstLevelModel(t_r=1.5,
                           noise_model='ar1',
                           hrf_model='spm',
                           drift_model='cosine',
                           high_pass=1./160,
                           signal_scaling=False,
                           minimize_memory=False)

Usually, we also want to include confounds computed during preprocessing (e.g., motion, global signal, etc.) as regressors of no interest. In our example, these were computed by `fmriprep` and can be found in `derivatives/fmriprep/sub-01/func/`. We can use `pandas` to inspect that file:

In [ ]:
confounds = pd.read_csv(join(data_dir, 'derivatives/sub-01/func/sub-01_task-languagelocalizer_desc-confounds_regressors.tsv'), 
                        delimiter='\t')
confounds

Now that we have specified the model, we can run it on the fMRI image

In [ ]:
# we are doing this for the sub-01, in this example
fmri_img = join(data_dir, 'derivatives/sub-01/func/sub-01_task-languagelocalizer_desc-preproc_bold.nii.gz')
fmri_glm = fmri_glm.fit(fmri_img, events, confounds)

One can inspect the design matrix (rows represent time, and columns contain the predictors).

In [ ]:
design_matrix = fmri_glm.design_matrices_[0]

Formally, we have taken the first design matrix, because the model is implictily meant to for multiple runs.

In [ ]:
from nilearn.plotting import plot_design_matrix
plot_design_matrix(design_matrix)
import matplotlib.pyplot as plt
plt.show()

The first column contains the expected reponse profile of regions which are sensitive to the "Finger" task. Let's plot this first column:

In [ ]:
plt.plot(design_matrix['language'])
plt.xlabel('scan')
plt.title('Expected Response for condition "language"')
plt.show()

### Detecting voxels with significant effects

To access the estimated coefficients (Betas of the GLM model) for each condition, we
created constrast with a single '1' in each of the task columns.

In [ ]:
from numpy import array
conditions = {
    'language': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
    'string': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
}

Let's look at it: plot the coefficients of the contrast, indexed by the names of the columns of the design matrix.

In [ ]:
from nilearn.plotting import plot_contrast_matrix
plot_contrast_matrix(conditions['language'], design_matrix=design_matrix)

Below, we compute the estimated effect. It is in BOLD signal unit,
but has no statistical guarantees, because it does not take into
account the associated variance.

In [ ]:
eff_map = fmri_glm.compute_contrast(conditions['language'],
                                    output_type='effect_size')

In order to get statistical significance, we form a `t-statistic`, and directly convert is into `z-scale`. The `z-scale` means that the values are scaled to match a standard Gaussian distribution (mean=`0`, variance=`1`), across voxels, if there were now effects in the data.

In [ ]:
z_map = fmri_glm.compute_contrast(conditions['language'],
                                  output_type='z_score')

Plot thresholded z scores map.

We display it on top of the average functional image of the series (could be the anatomical image of the subject). We use arbitrarily a threshold of `3.0` in `z-scale`. We'll see later how to use corrected thresholds. 

In [ ]:
from nilearn.image import mean_img
mean_img = mean_img(fmri_img)

from nilearn.plotting import plot_stat_map, plot_anat, plot_img, show, plot_glass_brain

plot_stat_map(z_map, bg_img=mean_img, threshold=3.0,
              #display_mode='z', cut_coords=3, 
              black_bg=True,
              title='language (Z>3)')
plt.show()

In [ ]:
plot_glass_brain(z_map, threshold=3.0, black_bg=True, plot_abs=False,
                 title='language (Z>3)')
plt.show()

### Statistical signifiance testing
One should worry about the statistical validity of the procedure: here we used an arbitrary threshold of 3.0 but the threshold should provide some guarantees on the risk of false detections (aka `type-1` errors in statistics). One
first suggestion is to **control the false positive rate** (`fpr`) at a certain level, e.g. `0.001`.

In [ ]:
from nilearn.glm.thresholding import threshold_stats_img
_, threshold = threshold_stats_img(z_map, alpha=.001, height_control='fpr')
print('Uncorrected p<0.001 threshold: %.3f' % threshold)
plot_stat_map(z_map, bg_img=mean_img, threshold=threshold,
              #display_mode='z', cut_coords=3, 
              black_bg=True,
              title='language (p<0.001)')
plt.show()

In [ ]:
plot_glass_brain(z_map, threshold=threshold, black_bg=True, plot_abs=False,
                 title='language (p<0.001)')
plt.show()

The example above is not corrected for **multiple comparisons**. After all, we are performing thousands of `t-tests` here (one for each voxel). A more conservative solution is to control the **family wise error** rate, i.e. the probability of making ony one false detection, say at `5%`. For that we use the so-called `Bonferroni correction`.

In [ ]:
_, threshold = threshold_stats_img(z_map, alpha=.05, height_control='bonferroni')
print('Bonferroni-corrected, p<0.05 threshold: %.3f' % threshold)
plot_stat_map(z_map, bg_img=mean_img, threshold=threshold,
              #display_mode='z', cut_coords=3, 
              black_bg=True,
              title='language (p<0.05, corrected)')
plt.show()

In [ ]:
plot_glass_brain(z_map, threshold=threshold, black_bg=True, plot_abs=False,
                 title='language (p<0.05, corrected)')
plt.show()

Finally people like to discard isolated voxels from these images. It is possible to generate a thresholded map with small clusters removed by providing a `cluster_threshold` argument. Here clusters smaller than `10` voxels will be discarded.

In [ ]:
clean_map, threshold = threshold_stats_img(
    z_map, alpha=.05, height_control='bonferroni', cluster_threshold=10)
plot_stat_map(clean_map, bg_img=mean_img, threshold=threshold,
              #display_mode='z', cut_coords=3, 
              black_bg=True, colorbar=False,
              title='language (p<0.05, corrected), clusters > 10 voxels')
plt.show()

In [ ]:
plot_glass_brain(clean_map, threshold=threshold, black_bg=True, plot_abs=False,
                 title='language (p<0.05, corrected), clusters > 10 voxels)')
plt.show()

### Reporting

Report the found positions in a table

In [ ]:
from nilearn.reporting import get_clusters_table
table = get_clusters_table(z_map, stat_threshold=threshold,
                           cluster_threshold=20)
print(table)

Using the computed `FirstLevelModel` and contrast information, we can quickly create a summary report.

In [ ]:
from nilearn.reporting import make_glm_report

report = make_glm_report(fmri_glm,
                         contrasts='language',
                         bg_img=mean_img
                         )

In [ ]:
report

## Performing statistical analyses on BIDS datasets
Even though model specification and running was comparably easy and straightforward, it can be even better. `Nilearn`'s `GLM` functionality actually enables you to define models for multiple participants through one function by leveraging the `BIDS` standard. More precisely, the function `first_level_from_bids` takes the same input arguments as `First_Level_model` (e.g. `t_r`, `hrf_model`, `high_pass`, etc.), but through defining the `BIDS raw` and `derivatives folder`, as well as a `task` and `space` label automatically extracts all information necessary to run `individual level models` and creates the `model` itself for all participants. 

### Obtain automatically `FirstLevelModel` objects and fit arguments
From the dataset directory we automatically obtain the `FirstLevelModel` objects
with their subject_id filled from the `BIDS` dataset. Moreover, we obtain
for each model a dictionary with `run_imgs`, `events` and `confounder` regressors
since in this case a `confounds.tsv` file is available in the `BIDS` dataset.
To get the first level models we only have to specify the dataset directory
and the task_label as specified in the file names.

In [ ]:
first_level_from_bids?

In [ ]:
from nilearn.glm.first_level import first_level_from_bids
task_label = 'languagelocalizer'
models, models_run_imgs, models_events, models_confounds = \
    first_level_from_bids(
        data_dir, task_label,
        img_filters=[('desc', 'preproc')])

### Quick sanity check on fit arguments

We just expect one run_img per subject.



In [ ]:
import os
print([os.path.basename(run) for run in models_run_imgs[0]])

The only confounds stored are regressors obtained from motion correction. As
we can verify from the column headers of the confounds table corresponding
to the only run_img present.



In [ ]:
print(models_confounds[0][0].columns)

During this acquisition the subject read blocks of sentences and
consonant strings. So these are our only two conditions in events.
We verify there are 12 blocks for each condition.



In [ ]:
print(models_events[0][0]['trial_type'].value_counts())

We can also inspect the `FirstLevelModel` parameters. 

In [ ]:
print(models[1])

### First level model estimation for all subjects
Now we simply fit each first level model and plot for each subject the
`contrast` that reveals the language network (language - string).
Notice that we can define a contrast using the names of the conditions
specified in the events dataframe.
Sum, subtraction and scalar multiplication are allowed.

Set the threshold as the `z-variate` with an uncorrected p-value of `0.001`.

In [ ]:
from scipy.stats import norm
p001_unc = norm.isf(0.001)

Prepare figure for concurrent plot of individual maps.



In [ ]:
from nilearn import plotting
import matplotlib.pyplot as plt

models_fitted = [] 

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(8, 4.5))
model_and_args = zip(models, models_run_imgs, models_events, models_confounds)
for midx, (model, imgs, events, confounds) in enumerate(model_and_args):
    # fit the GLM
    model.fit(imgs, events, confounds)
    
    models_fitted.append(model)
    
    # compute the contrast of interest
    zmap = model.compute_contrast('language-string')
    plotting.plot_glass_brain(zmap, colorbar=False, threshold=p001_unc,
                              title=('sub-' + model.subject_label),
                              axes=axes[int(midx / 5), int(midx % 5)],
                              plot_abs=False, display_mode='x')
fig.suptitle('subjects z_map language network (unc p<0.001)')
plotting.show()

That looks about right. However, let's also check the `design matrix`

In [ ]:
from nilearn.plotting import plot_design_matrix
plot_design_matrix(models_fitted[0].design_matrices_[0])

and `contrast matrix`.

In [ ]:
plot_contrast_matrix('language', models_fitted[0].design_matrices_[0])
plt.show()

plot_contrast_matrix('language - string', models_fitted[0].design_matrices_[0])
plt.show()

### Second level model estimation
We just have to provide the list of fitted `FirstLevelModel` objects
to the `SecondLevelModel` object for estimation. We can do this because
all subjects share a similar design matrix (same variables reflected in
column names).

In [ ]:
from nilearn.glm.second_level import SecondLevelModel
second_level_input = models

Note that we apply a smoothing of `8mm`.



In [ ]:
second_level_model = SecondLevelModel(smoothing_fwhm=8.0)
second_level_model = second_level_model.fit(second_level_input)

Computing contrasts at the second level is as simple as at the first level.
Since we are not providing confounders we are performing a `one-sample test`
at the second level with the images determined by the specified first level
contrast.

In [ ]:
zmap = second_level_model.compute_contrast(
    first_level_contrast='language-string')

The group level contrast reveals a left lateralized fronto-temporal
language network.



In [ ]:
plotting.plot_glass_brain(zmap, colorbar=True, threshold=p001_unc,
                          title='Group language network (unc p<0.001)',
                          plot_abs=False, display_mode='x')
plotting.show()

### A summary report
And now we can create a summary report.

In [ ]:
from nilearn.reporting import make_glm_report

report = make_glm_report(model=model,
                         cluster_threshold = 10,
                         contrasts='language -string',
                         display_mode = 'ortho'
                         )

In [ ]:
report